# Data management

In this notebook, we will work with the following:

- Reading and cleaning data with `pandas`.
- Web scraping.
- API access.
- Merging and querying with `pandas`.

In [1]:
import urllib

import pandas as pd
from bs4 import BeautifulSoup
from nytimesarticle import articleAPI

# Reading and cleaning data

`pandas` supports a number of formats that we often find ourselves using.
For example, I often use data in the Stata `dta` and SAS `sas7bdat` formats.
In particular, if you find yourself putting full datasets from WRDS (especially the ones that are not accessible with web forms), you will end up using the SAS format.

`pandas` also handles formats like Excel `xlsx`, comma separated values `csv` (and, indeed, nearly any delimited file), and fixed width data.
The acquisition database, SDC Platinum, has a somewhat unreliable Excel output feature, and the `pandas` fixed width format reader takes nearly all of the pain out of reading in data exported that way.

Note: `pandas` can also write many of the formats that it can read.
A notable exception is `sas7bdat` because it is proprietary and undocumented.
The reader was written with some clever reverse engineering, but writing a valid file is difficult and probably not coming in the future (see [Github issue](https://github.com/pandas-dev/pandas/issues/13031)).
An easy workaround is using the SAS open format `xpt` or `csv`.

In [2]:
# Stata data
firmyear = pd.read_stata('./data/firmyear.dta')
firmyear.head()

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017


You are likely familiar with a number of data cleaning issues.
However, you may not yet know how to map on what you know in another program to Python.
The pandas documentation has a number of comparison references, including [R](https://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html), [Stata](https://pandas.pydata.org/pandas-docs/stable/comparison_with_stata.html) and [SAS](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sas.html).

Some brief examples are below.

In [3]:
firmyear.dtypes

count_of_employees    object
name                  object
year                  object
dtype: object

In [4]:
firmyear['year'] = firmyear['year'].astype('int')
firmyear['count_of_employees'] = firmyear['count_of_employees'].astype('int')

# Note, a more general version would be:
# cols = firmyear.columns.drop('name')
# firmyear[cols] = firmyear[cols].apply(pd.to_numeric, errors='coerce')

In [5]:
firmyear

,count_of_employees,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


In [6]:
firmyear.dtypes

count_of_employees     int64
name                  object
year                   int64
dtype: object

In [7]:
# An example of using dictionaries.
_COLUMNS = {
    'count_of_employees': 'size_emp'
}

firmyear = firmyear.rename(columns=_COLUMNS)

In [8]:
firmyear

,size_emp,name,year
0,114,Microsoft,2016
1,124,Microsoft,2017
2,131,Microsoft,2018
3,72,Google,2016
4,74,Google,2017
5,76,Google,2018


In [9]:
# We can do per-group things like calculating differences.
firmyear['size_emp_change'] = firmyear.groupby(firmyear['name'])['size_emp'].diff()

In [10]:
firmyear

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0
5,76,Google,2018,2.0


# Web scraping

One helpful way of gathering text data is web scraping.
We usually do this in three steps:

1. Retrieve the pages with information we want.
1. Extract the data from the pages.
1. Clean and save the resulting data.

Let's walk through an example of getting press releases from the [Microsoft website](https://news.microsoft.com/category/press-releases/).

I often prefer to work out of order as follows:

1. Figure out how to extract data from one page that has the data.
1. Then, figure out how to automate getting the pages of interest.
1. Run those pages through the procedure in step 1.
1. Clean and save.

This has the benefit of solving what is usually the hardest problem first.

## Important note

As you'll see, the difficulty ramps up a lot here.
Web scraping is easily a full day topic on its own.
Hence, I have two main goals for you:

1. Get a sense of the logic and the process in solving the problem. This is a good start if you want to learn it yourself.
1. Understand what is feasible and achievable. This helps whether you do it yourself or farm it out (and there's a ready talent pool for this).

## Read one page

This is the hardest part.

In [11]:
pr_url_1 = 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/'
pr_req_1 = urllib.request.Request(pr_url_1)
pr_page_1 = urllib.request.urlopen(pr_req_1).read()

In [12]:
pr_soup_1 = BeautifulSoup(pr_page_1)

In [13]:
pr_soup_1.find('meta', property='article:published_time')

<meta content="2018-10-04T13:00:35+00:00" property="article:published_time"/>

In [14]:
pr_soup_1.find('meta', property='article:published_time')['property']

'article:published_time'

In [15]:
pr_soup_1.find('meta', property='article:published_time')['content']

'2018-10-04T13:00:35+00:00'

In [16]:
# List of meta tags to get.
# Note: when in doubt, get everything you might possibly use.
#       It's easier to drop stuff than to re-scrape everything.

_METAS = [
    'article:published_time',
    'article:modified_time',
    'og:title',
    'og:description',
    'og:updated_time',
    'og:url',
    'article:section'
]

In [17]:
pr_data_1 = {}
for meta in _METAS:
    prop = pr_soup_1.find('meta', property=meta)['property']
    content = pr_soup_1.find('meta', property=meta)['content']
    pr_data_1.update({prop: content})

In [18]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases'}

In [19]:
pr_soup_1.find('div', {'class': 'entry-content m-blog-content'}).find('h3').string

'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally'

In [20]:
pr_data_1['h3'] = pr_soup_1.find('div', 
                                 {'class': 'entry-content m-blog-content'}
                                ).find('h3').string

In [21]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

In [22]:
pr_soup_1.find('div', 
               {'class': 'entry-content m-blog-content'}
              ).find_all('p')

[<p><strong>REDMOND, Wash. </strong><strong>—</strong> <strong>Oct. 4, 2018</strong> <strong>—</strong> On Thursday, <a href="https://rdlcom.com/">Redline Communications</a> (TSX:RDL) and <a href="https://www.microsoft.com/en-us/">Microsoft Corp.</a> announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a leader in private wireless networks, will provide its Virtual Fiber™ radio technology in the TV White Space band to Microsoft Airband Initiative partners. Together, Redline and Microsoft’s partnership will help make broadband internet more affordable and accessible to unserved and underserved customers in rural areas in the United States and globally.</p>,
 <p>New cloud services and other technologies make broadband connectivity a necessity to start and grow a small business and to take advantage of advances in agriculture, telemedicine and education. It is a vital part of 21st century infrastructure. Yet, more than 19.4 m

In [23]:
# This is a little gnarly.
pr_data_1['body'] = '\n\n'.join(
                        [i.text for i in pr_soup_1.find(
                            'div', 
                            {'class': 'entry-content m-blog-content'}
                            ).find_all('p')])

In [24]:
pr_data_1

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

# Automate our one page work.

This is fairly easy. We have the code for it already.
We just need to wrap it in a function.

**Note:** I'm using an `if` statement to check whether these properties exist, and guarding against the case where they don't.
I did this iteratively while building this content, because I noticed (from errors) that many press releases do noe have modification dates.

In [25]:
def get_data_from_soup(soup):
    data = {}
    for meta in _METAS:
        if soup.find('meta', property=meta) is not None:
            prop = soup.find('meta', property=meta)['property']
        if soup.find('meta', property=meta) is not None:
            content = soup.find('meta', property=meta)['content']
        if prop is not None and content is not None:
            data.update({prop: content})
    
    data['h3'] = soup.find('div', 
                           {'class': 'entry-content m-blog-content'}
                          ).find('h3').string
    
    data['body'] = '\n\n'.join(
                        [i.text for i in soup.find(
                            'div', 
                            {'class': 'entry-content m-blog-content'}
                            ).find_all('p')])
    
    return data

In [26]:
# Notice how easy this is once we make a function.
get_data_from_soup(pr_soup_1)

{'article:published_time': '2018-10-04T13:00:35+00:00',
 'article:modified_time': '2018-10-04T14:43:59+00:00',
 'og:title': 'Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions - Stories',
 'og:description': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the U.S. and globally REDMOND, Wash. — Oct. 4, 2018 — On Thursday, Redline Communications (TSX:RDL) and Microsoft Corp. announced a new partnership that will help address the rural broadband gap using TV White Space technology. Redline, a […ellipsis]',
 'og:updated_time': '2018-10-04T14:43:59+00:00',
 'og:url': 'https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'article:section': 'Press Releases',
 'h3': 'The partnership will help make broadband more affordable and accessible for unserved communities in rural areas of the 

## Read many pages

Now we need to get the URLs for all of the pages we want.

In [27]:
many_pr_url_1 = 'https://news.microsoft.com/category/press-releases/'
many_pr_req_1 = urllib.request.Request(many_pr_url_1)
many_pr_page_1 = urllib.request.urlopen(many_pr_req_1).read()
many_pr_soup_1 = BeautifulSoup(many_pr_page_1)

In [28]:
# Almost, but note the ones at the bottom.
many_pr_soup_1.find('section', id='primary').find_all('a')

[<a class="f-post-link c-heading-6 m-chevron" href="https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/" ms.title="Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions" rel="bookmark">
 		Redline Communications and Microsoft announce partnership to lower the cost of TV White Space solutions	</a>,
 <a class="f-post-link c-heading-6 m-chevron" href="https://news.microsoft.com/2018/09/24/adobe-microsoft-and-sap-announce-the-open-data-initiative-to-empower-a-new-generation-of-customer-experiences/" ms.title="Adobe, Microsoft and SAP announce the Open Data Initiative to empower a new generation of customer experiences" rel="bookmark">
 		Adobe, Microsoft and SAP announce the Open Data Initiative to empower a new generation of customer experiences	</a>,
 <a class="f-post-link c-heading-6 m-chevron" href="https://news.microsoft.com/2018/09/24/microsoft-

In [29]:
# Almost, but note the ones at the bottom.
articles = many_pr_soup_1.find('section', id='primary').find_all('article')
links = [i.find('a')['href'] for i in articles]
links

['https://news.microsoft.com/2018/10/04/redline-communications-and-microsoft-announce-partnership-to-lower-the-cost-of-tv-white-space-solutions/',
 'https://news.microsoft.com/2018/09/24/adobe-microsoft-and-sap-announce-the-open-data-initiative-to-empower-a-new-generation-of-customer-experiences/',
 'https://news.microsoft.com/2018/09/24/microsoft-fortifies-security-and-brings-ai-to-the-masses-at-ignite-2018/',
 'https://news.microsoft.com/2018/09/20/shell-expands-strategic-collaboration-with-microsoft-to-drive-industry-transformation-and-innovation/',
 'https://news.microsoft.com/2018/09/18/microsoft-announces-quarterly-dividend-increase/',
 'https://news.microsoft.com/2018/09/13/network-business-systems-and-microsoft-announce-agreement-to-deliver-broadband-internet-to-rural-communities-in-illinois-iowa-and-south-dakota/',
 'https://news.microsoft.com/2018/08/08/agile-networks-and-microsoft-announce-agreement-to-deliver-broadband-internet-access-to-rural-communities-in-ohio/',
 'https

In [30]:
many_pr_links_1 = links.copy()

## Automate getting links and data from each

In [31]:
# We need to turn links into soup objects a lot, so let's make a function.
def link_to_soup(link):
    req = urllib.request.Request(link)
    page = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(page)
    return soup
    
def get_links_from_link_page(link_page):
    soup = link_to_soup(link_page)
    articles = soup.find('section', id='primary').find_all('article')
    links = [i.find('a')['href'] for i in articles]
    return links

def get_data_from_links(links):
    data_list = []
    for link in links:
        soup = link_to_soup(link)
        data_list.append(get_data_from_soup(soup))
        
    return data_list


In [32]:
pd.DataFrame(get_data_from_links(many_pr_links_1))

,article:modified_time,article:published_time,article:section,body,h3,og:description,og:title,og:updated_time,og:url
0,2018-10-04T14:43:59+00:00,2018-10-04T13:00:35+00:00,Press Releases,"REDMOND, Wash. — Oct. 4, 2018 — On Thursday, R...",The partnership will help make broadband more ...,The partnership will help make broadband more ...,Redline Communications and Microsoft announce ...,2018-10-04T14:43:59+00:00,https://news.microsoft.com/2018/10/04/redline-...
1,2018-09-24T16:11:34+00:00,2018-09-24T13:30:14+00:00,Press Releases,"\nShantanu Narayen, CEO, Adobe (left), Satya N...",Industry leaders team up to help customers con...,Industry leaders team up to help customers con...,"Adobe, Microsoft and SAP announce the Open Dat...",2018-09-24T16:11:34+00:00,https://news.microsoft.com/2018/09/24/adobe-mi...
2,NaN,2018-09-24T13:01:27+00:00,Press Releases,"ORLANDO, Fla. — Sept. 24, 2018 — Monday at Ign...","The company also demonstrated how AI, IoT and ...","The company also demonstrated how AI, IoT and ...",Microsoft fortifies security and brings AI to ...,NaN,https://news.microsoft.com/2018/09/24/microsof...
3,2018-09-20T15:19:45+00:00,2018-09-20T13:00:12+00:00,Press Releases,"REDMOND, Wash., and HOUSTON — Sept. 20, 2018 —...",Shell appoints C3 IoT and Microsoft Azure as i...,Shell appoints C3 IoT and Microsoft Azure as i...,Shell expands strategic collaboration with Mic...,2018-09-20T15:19:45+00:00,https://news.microsoft.com/2018/09/20/shell-ex...
4,NaN,2018-09-18T22:35:54+00:00,Press Releases,"REDMOND, Wash. — Sept. 18, 2018 — Microsoft Co...","Annual shareholders meeting set for Nov. 28, 2018","Annual shareholders meeting set for Nov. 28, 2...",Microsoft announces quarterly dividend increas...,NaN,https://news.microsoft.com/2018/09/18/microsof...
5,NaN,2018-09-13T13:00:07+00:00,Press Releases,"REDMOND, Wash. — Sept. 13, 2018 — On Thursday,...",The partnership will benefit hundreds of thous...,The partnership will benefit hundreds of thous...,Network Business Systems and Microsoft announc...,NaN,https://news.microsoft.com/2018/09/13/network-...
6,NaN,2018-08-08T13:00:15+00:00,Press Releases,"CANTON, OH (AUGUST 8, 2018) – Today, Agile Net...",The agreement will leverage underutilized infr...,The agreement will leverage underutilized infr...,Agile Networks and Microsoft announce agreemen...,NaN,https://news.microsoft.com/2018/08/08/agile-ne...
7,NaN,2018-08-01T13:02:21+00:00,Press Releases,"REDMOND, Wash. — Aug. 1, 2018 — On Wednesday, ...",The grant will help expand Numbers4Health’s te...,The grant will help expand Numbers4Health’s te...,Microsoft awards grant to Numbers4Health to ex...,NaN,https://news.microsoft.com/2018/08/01/microsof...
8,NaN,2018-08-01T13:02:02+00:00,Press Releases,"REDMOND, Wash. — Aug. 1, 2018 — On Wednesday, ...",The grant will provide broadband access and te...,The grant will provide broadband access and te...,Microsoft awards grant to Tribal Digital Villa...,NaN,https://news.microsoft.com/2018/08/01/microsof...
9,NaN,2018-08-01T13:01:18+00:00,Press Releases,"REDMOND, Wash. — Aug. 1, 2018 — On Wednesday, ...",The grant will help expand Skylark Wireless’ n...,The grant will help expand Skylark Wireless’ n...,Microsoft awards grant to Skylark Wireless to ...,NaN,https://news.microsoft.com/2018/08/01/microsof...


**Note**: for running time reasons, we're not going to make a multi-links-page version, but note that there's a next page link at the bottom of those pages that can be extracted to build that:

```html
<a href="/category/press-releases/page/2/?paged=3" 
   class="c-glyph x-hidden-focus" 
   aria-label="Go to next page" ms.title="Next Page">
```

# APIs

As we're about to see, it's really nice when sites help us out.

We'll be using the New York Times API, and you'll need a key to use it. You can get one [here](https://developer.nytimes.com/signup), and it will show up by email (very quickly).

In [33]:
# You will replace REDACTED with the key you got in email.
# It's probably obvious, but don't share it.
nyt_api = articleAPI('REDACTED')

msft_articles = nyt_api.search(q='Microsoft', begin_date=20180801)

In [34]:
# Trust me that the JSON for even a short period is very long,
# thus the immediate dataframe processing.
msft_nyt = pd.DataFrame(pd.io.json.json_normalize(msft_articles['response']['docs']))
msft_nyt.head()

,_id,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,headline.print_headline,...,print_page,pub_date,score,section_name,snippet,source,type_of_material,uri,web_url,word_count
0,5bae3952068401528a2dd72e,REUTERS,By REUTERS,[],article,None,None,Volkswagen Partners With Microsoft in Cloud Push,None,Volkswagen Partners With Microsoft in Cloud Push,...,NaN,2018-09-28T14:23:13+0000,308.69720,NaN,German automaker Volkswagen AG will use Micros...,Reuters,News,nyt://article/f5d24531-d0b6-566e-a0bb-76fd8908...,https://www.nytimes.com/reuters/2018/09/28/bus...,99
1,5ba8e824068401528a2dca0f,REUTERS,By REUTERS,[],article,None,None,"SAP, Microsoft and Adobe Announce Data Alliance",None,"SAP, Microsoft and Adobe Announce Data Alliance",...,NaN,2018-09-24T13:35:30+0000,290.06796,NaN,"Business software companies SAP, Microsoft and...",Reuters,News,nyt://article/53a342e8-245a-57e5-8f5a-0f4744a2...,https://www.nytimes.com/reuters/2018/09/24/tec...,198
2,5bb295c6068401528a2ddeeb,REUTERS,By REUTERS,[],article,None,None,Microsoft Co-Founder Paul Allen Treated Again ...,None,Microsoft Co-Founder Paul Allen Treated Again ...,...,NaN,2018-10-01T21:46:45+0000,260.16263,NaN,Microsoft Corp co-founder Paul Allen said on M...,Reuters,News,nyt://article/2f19d5d8-3def-5d8d-9ba8-519385d2...,https://www.nytimes.com/reuters/2018/10/01/us/...,273
3,5b7b8e47068401528a2d4ea0,None,By DAVID E. SANGER and SHEERA FRENKEL,"[{'firstname': 'David', 'middlename': 'E.', 'l...",article,None,None,New Russian Hacking Targeted Republican Groups...,None,Russian Hackers Broaden Attacks To Conservatives,...,1,2018-08-21T04:00:04+0000,224.41266,Politics,Microsoft was surprised not that recent “spear...,The New York Times,News,nyt://article/13a88310-6a74-5646-89c2-f86df163...,https://www.nytimes.com/2018/08/21/us/politics...,1256
4,5b7bde7f068401528a2d4f4a,None,By CHRIS STANFORD,"[{'firstname': 'Chris', 'middlename': None, 'l...",article,None,None,"Microsoft, Primary Election, E.P.A.: Your Tues...",None,,...,NaN,2018-08-21T09:42:21+0000,210.07143,NaN,Here’s what you need to know to start your day.,The New York Times,briefing,nyt://article/b6f14469-da14-5d59-95e9-63326c56...,https://www.nytimes.com/2018/08/21/briefing/mi...,1354


In [35]:
# While we're at it, let's add Microsoft's ticker.
# We'd usually add an identifier when getting query results.
msft_nyt['id_ticker'] = 'msft'

In [36]:
# We should also clean up the publication date.
type(msft_nyt['pub_date'][0])

str

In [37]:
msft_nyt['pub_date'] = pd.to_datetime(msft_nyt['pub_date'])
type(msft_nyt['pub_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [38]:
# We're going to look at this a bit later, so let's save it.
msft_nyt.to_csv('./data/msft_nyt.csv', index=False)

# Commercial databases

This is a more difficult topic to cover in a hands-on workshop (especially one with  openly-distributed materials) because of copyright.
They tend to sort out into a couple of types:

1. Those that come in tabular formats and simply require cleaning.
2. Others (like LexisNexis and Factiva) that come in semi-structured form and require extensive parsing. 

For the second type, it's best to either recruit one of the few coauthors with this skillset (and perhaps even a written or partially written implementation) or hiring a programmer or student who can write one.
It's worth noting that it's not all that hard to get a reasonable implementation written, but going from 90 percent parsing accuracy to 99 percent to 99.9+ percent is difficult, painstaking work.

With that in mind, feel free to ask questions about this topic during the workshop.
If time permits, I'll demonstrate some non-sharable stuff.

# Merging

Like other software, `pandas` is great and merging data, and it as some conveniences not found in most other software.

Let's work through a simple example to see it in action.

In [39]:
# Remember our firm year data.
firmyear.head()

,size_emp,name,year,size_emp_change
0,114,Microsoft,2016,NaN
1,124,Microsoft,2017,10.0
2,131,Microsoft,2018,7.0
3,72,Google,2016,NaN
4,74,Google,2017,2.0


In [40]:
stock = pd.read_csv('./data/stock.csv')
stock.head()

,price,tic,yr
0,86.13,msft,2018
1,62.79,msft,2017
2,54.32,msft,2016


What we'd like to do is merge in those Microsoft stock prices from the beginning of those years.
It's a bit contrived for an example, but it mirrors a lot of real world work.

While we know that Microsoft's ticker is MSFT, there's no way for `pandas` to know that without help.
So, to help, we'll make a lookup table using a dictionary.

In [41]:
lookup = {
    'Microsoft': 'MSFT',
    'Google': 'GOOG'
}

In [42]:
firmyear['id_ticker'] = firmyear['name'].map(lookup)
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,MSFT
1,124,Microsoft,2017,10.0,MSFT
2,131,Microsoft,2018,7.0,MSFT
3,72,Google,2016,NaN,GOOG
4,74,Google,2017,2.0,GOOG


In [43]:
# Let's make that lowercase.
firmyear['id_ticker'] = firmyear['id_ticker'].str.lower()
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker
0,114,Microsoft,2016,NaN,msft
1,124,Microsoft,2017,10.0,msft
2,131,Microsoft,2018,7.0,msft
3,72,Google,2016,NaN,goog
4,74,Google,2017,2.0,goog


In Stata, we would have another problem, namely that our column names for merging do not match.
With `pandas`, that's not a problem.

In [44]:
firmyear = firmyear.merge(stock, how='left', 
                          left_on=['id_ticker', 'year'],
                          right_on=['tic', 'yr'],
                          validate='1:1')

In [45]:
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN
4,74,Google,2017,2.0,goog,NaN,NaN,NaN


# Querying

When working with content data, we often need to do some sort of a query to aggregate data that is interesting to us.

For example, let's add the average word count of articles from our NYT data to our firmyear data.
We're only going to have results for 2018, as that's all the data we pulled.

In [46]:
_AGG = {
    'word_count': ['mean', 'sum']
}

def query_docs(data, ticker, year):
    summary = (data[(data['id_ticker'] == ticker) & 
                    (data['pub_date'].dt.year == year)].agg(_AGG)
                     .T.reset_index(drop=True))
    summary['id_ticker'] = ticker
    summary['year'] = year
    summary = summary.rename(columns={'mean': 'wc_mean', 'sum': 'wc_sum'})
    return summary

In [47]:
results = pd.DataFrame()
for index, row in firmyear.loc[:, ['id_ticker', 'year']].iterrows():
    results = results.append(query_docs(msft_nyt, row['id_ticker'], row['year']))

In [48]:
results

,wc_mean,wc_sum,id_ticker,year
0,NaN,0.0,msft,2016
0,NaN,0.0,msft,2017
0,579.1,5791.0,msft,2018
0,NaN,0.0,goog,2016
0,NaN,0.0,goog,2017
0,NaN,0.0,goog,2018


In [49]:
firmyear = firmyear.merge(results, how='left',
                          on=['id_ticker', 'year'],
                          validate='1:1')
firmyear.head()

,size_emp,name,year,size_emp_change,id_ticker,price,tic,yr,wc_mean,wc_sum
0,114,Microsoft,2016,NaN,msft,54.32,msft,2016.0,NaN,0.0
1,124,Microsoft,2017,10.0,msft,62.79,msft,2017.0,NaN,0.0
2,131,Microsoft,2018,7.0,msft,86.13,msft,2018.0,579.1,5791.0
3,72,Google,2016,NaN,goog,NaN,NaN,NaN,NaN,0.0
4,74,Google,2017,2.0,goog,NaN,NaN,NaN,NaN,0.0
